In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]='2'

os.environ["WANDB_API_KEY"]='a4ea3766f1f461414ce8e6b8211deb96e05bd366'

In [2]:
from generation_module import *
from graph_utils import * 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import wandb
from pytorch_lightning.loggers import WandbLogger


/home/mbugueno/anaconda3/envs/py38_torch111/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import argparse
import csv
import dgl
import itertools
import numpy as np
import random, re, os, nltk
import sys, random
import torch
import torch.nn.functional as F
import tqdm
import time, datetime
import word2vec

from itertools import compress
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from transformers import logging
logging.set_verbosity_error()

In [4]:
from wandb_loader_TLGCN import *

/home/mbugueno/anaconda3/envs/py38_torch111/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


In [7]:
def train_and_evaluate(project_name, dataset, bs, ngram, num_exec, num_epoch, embedding_path, dim_emb, bert=False, dropout = 0.2, edges = True, bar = False):
      
    name = 'temp_model'
    model, total_timec, total_timetrain = train(project_name, ngram, name, bar, dropout, embedding_path, dim_emb, dataset, bs, execn=num_exec, n_epochs=num_epoch, bert=bert, is_cuda=True, edges=edges)
    
    test_acc, test_f1, test_f1ma, total_timetest = test(ngram, embedding_path, model, dataset, bs, execn=num_exec)

    np.set_printoptions(precision=3)
    print('Test acc: %.3f' % test_acc.numpy())
    print('Test F1:', test_f1)
    print('Test F1-ma: %.3f' % test_f1ma)
    
    return test_acc, test_f1, test_f1ma, total_timec, total_timetrain+total_timetest

In [8]:
def run_loop(path_metrics, project_name, file_to_save, dataset, bs, ngram, num_execs, num_epoch, embedding_path, dim_emb, bert=False):
    np.set_printoptions(precision=3)
    acc_tests=[]
    f1_tests=[]
    f1ma_tests=[]
    times_create=0.0
    times_run=0.0
    start = time.time()
    np.set_printoptions(precision=3)
    
    with open(path_metrics+file_to_save+'.txt', 'a') as f:  
    
        for i in range(num_execs):
            print ("\nRunning execution", i)
            test_acc, test_f1, test_f1ma, time_create, time_run = train_and_evaluate(project_name, dataset, bs, ngram, i, num_epoch, embedding_path, dim_emb, bert=bert)

            print("Graph creation time "+str(i)+": "+ str(time_create), file=f)
            print("Graph creation time "+str(i)+": "+ str(time_create))
            print("Running time "+str(i)+": "+ str(time_run), file=f)
            print("Running time "+str(i)+": "+ str(time_run))
            acc_tests.append(test_acc)
            f1_tests.append(test_f1)
            f1ma_tests.append(test_f1ma)
            times_create+=time_create
            times_run+=time_run

        print ("\n************************************************", file=f)
        print ("RESULTS FOR N_GRAM:", ngram, "EMB DIM_FEATURES:", embedding_path, file=f)    
        print ("Test Acc: %.3f"% np.mean(np.asarray(acc_tests)), "-- std: %.3f" % np.std(np.asarray(acc_tests)), file=f)
        print ("Test F1-macro: %.3f"%np.mean(np.asarray(f1ma_tests)), "-- std: %.3f" % np.std(np.asarray(f1ma_tests)), file=f)
        print ("Test F1 per class:", np.mean(np.asarray(f1_tests), axis=0), file=f)
        print ("Mean Creation Time:", times_create/num_execs, file=f)
        print ("Mean Running Time:", times_run/num_execs, file=f)
        print ("Mean Total Execution Time:",  times_create/num_execs + times_run/num_execs , file=f)
        print ("************************************************\n\n", file=f)
        print ("\n************************************************")
        print ("RESULTS FOR N_GRAM:", ngram, "EMB DIM_FEATURES:", embedding_path)  
        print ("Test Acc: %.3f"% np.mean(np.asarray(acc_tests)), "-- std: %.3f" % np.std(np.asarray(acc_tests)))
        print ("Test F1-macro: %.3f"%np.mean(np.asarray(f1ma_tests)), "-- std: %.3f" % np.std(np.asarray(f1ma_tests)))
        print ("Test F1 per class:", np.mean(np.asarray(f1_tests), axis=0))
        print ("Mean Creation Time:", times_create/num_execs)
        print ("Mean Running Time:", times_run/num_execs)        
        print ("Mean Total Execution Time:",  times_create/num_execs + times_run/num_execs)
        print ("************************************************\n\n")
        
        emb_method=embedding_path.split('-')[0]
        
    return acc_tests, f1_tests, f1ma_tests

### Prepare data

In [ ]:
allowed=['app_reviews', 'dbpedia_14', 'imdb', 'bbc', 'hyperpartisan_news_detection']

for dataset in allowed:
    print ("___________________________________")
    print ("Loading dataset", dataset)
    df_train, df_test = dataset_to_dataframe(dataset)
    stem_corpus(dataset, df_train, num=7000, n_short=3, mode="training")
    stem_corpus(dataset, df_test, num=3000, n_short=3, mode="test")

cut_datasets(0.8, allowed)

## Run Text Level GCN - Escalability

#### 1-gram

In [10]:
embedding_path="word2vec-google-news-300" # bert-base-uncased, glove-wiki-gigaword-300
dataset="app_reviews" ## dbpedia_14, imdb, bbc, hyperpartisan_news_detection 

project_name="ESC_TLGCN_APP"  ## -- Track model performance by using Weights & Biases
file_to_save= "WAND_TextLevelGCN_"+dataset
bs= 16
ng= 1

acc_tests, f1ma_tests, f1_tests = run_loop(project_name, file_to_save, dataset, bs, ng, 3, 100, embedding_path, 300, bert=False)


Running execution 0


acc,▁
loss,▁
acc,0.59594
loss,1.35029


Loading data helper in training mode.
Creating a new model.


/home/mbugueno/graph-based_TR/TextLevelGCN/wandb_loader_TLGCN.py:290: RuntimeWarning: invalid value encountered in double_scalars
  pmi_matrix[i, j] = np.log(pair_count_matrix[i, j] / (word_count[i] * word_count[j]))
/home/mbugueno/graph-based_TR/TextLevelGCN/wandb_loader_TLGCN.py:290: RuntimeWarning: divide by zero encountered in log
  pmi_matrix[i, j] = np.log(pair_count_matrix[i, j] / (word_count[i] * word_count[j]))


Loading word2vec-google-news-300
Model(
  (seq_edge_w): Embedding(206452, 1)
  (node_hidden): Embedding(1177, 300)
  (dropout): Dropout(p=0.2, inplace=False)
  (activation): ReLU()
  (Linear): Linear(in_features=300, out_features=5, bias=True)
)


LOS PARAMS
RESULTA 561057

Cuda available


...

Epoch:  20 -- Iter:  7000, Train Loss:  0.35, Train Acc:88.78% Val Acc:60.10%, Val_F1ma:32.15%, Time:0:04:07


/home/mbugueno/anaconda3/envs/py38_torch111/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


Epoch:  20 -- Iter:  7200, Train Loss:  0.38, Train Acc:87.81% Val Acc:58.79%, Val_F1ma:32.56%, Time:0:04:14


/home/mbugueno/anaconda3/envs/py38_torch111/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'
/home/mbugueno/anaconda3/envs/py38_torch111/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


Test acc: 0.639
Test F1: [0.5   0.202 0.127 0.163 0.804]
Test F1-ma: 0.359
Graph creation time 2: 48.97498631477356
Running time 2: 265.3986954689026

************************************************
RESULTS FOR N_GRAM: 1 EMB DIM_FEATURES: word2vec-google-news-300
Test Acc: 0.650 -- std: 0.011
Test F1-macro: 0.349 -- std: 0.007
Test F1 per class: [0.507 0.192 0.082 0.155 0.809]
Mean Creation Time: 51.9385179678599
Mean Running Time: 258.7221693992615
Mean Total Execution Time: 310.6606873671214
************************************************




In [10]:
embedding_path="word2vec-google-news-300"
dataset="app_reviews"
project_name="ESC_TLGCN_APP"
file_to_save= "WAND_TextLevelGCN_"+dataset
bs= 16
ng= 2

acc_tests, f1ma_tests, f1_tests = run_loop(project_name, file_to_save, dataset, bs, ng, 3, 100, embedding_path, 300, bert=False)


Running execution 0


wandb: Currently logged in as: mbuguenop (gnn_tc). Use `wandb login --relogin` to force relogin


Loading data helper in training mode.
Creating a new model.


/home/mbugueno/graph-based_TR/TextLevelGCN/wandb_loader_TLGCN.py:290: RuntimeWarning: invalid value encountered in double_scalars
  pmi_matrix[i, j] = np.log(pair_count_matrix[i, j] / (word_count[i] * word_count[j]))
/home/mbugueno/graph-based_TR/TextLevelGCN/wandb_loader_TLGCN.py:290: RuntimeWarning: divide by zero encountered in log
  pmi_matrix[i, j] = np.log(pair_count_matrix[i, j] / (word_count[i] * word_count[j]))


Loading word2vec-google-news-300
Model(
  (seq_edge_w): Embedding(206452, 1)
  (node_hidden): Embedding(1177, 300)
  (dropout): Dropout(p=0.2, inplace=False)
  (activation): ReLU()
  (Linear): Linear(in_features=300, out_features=5, bias=True)
)


LOS PARAMS
RESULTA 561057

Cuda available


...

Epoch:  44 -- Iter: 15200, Train Loss:  0.32, Train Acc:89.38% Val Acc:59.30%, Val_F1ma:32.15%, Time:0:10:37


/home/mbugueno/anaconda3/envs/py38_torch111/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


Epoch:  44 -- Iter: 15400, Train Loss:  0.34, Train Acc:89.19% Val Acc:59.52%, Val_F1ma:32.25%, Time:0:10:45


/home/mbugueno/anaconda3/envs/py38_torch111/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'
/home/mbugueno/anaconda3/envs/py38_torch111/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


Test acc: 0.595
Test F1: [0.469 0.101 0.1   0.188 0.776]
Test F1-ma: 0.327
Graph creation time 2: 62.43692231178284
Running time 2: 657.8235049247742

************************************************
RESULTS FOR N_GRAM: 2 EMB DIM_FEATURES: word2vec-google-news-300
Test Acc: 0.611 -- std: 0.014
Test F1-macro: 0.341 -- std: 0.010
Test F1 per class: [0.483 0.116 0.134 0.185 0.787]
Mean Creation Time: 58.2199866771698
Mean Running Time: 524.5726595719656
Mean Total Execution Time: 582.7926462491354
************************************************




In [9]:
embedding_path="word2vec-google-news-300"
dataset="app_reviews"
project_name="ESC_TLGCN_APP"
file_to_save= "WAND_TextLevelGCN_"+dataset
bs= 16
ng= 3

acc_tests, f1ma_tests, f1_tests = run_loop(project_name, file_to_save, dataset, bs, ng, 3, 100, embedding_path, 300, bert=False)


Running execution 0


wandb: Currently logged in as: mbuguenop (gnn_tc). Use `wandb login --relogin` to force relogin


Loading data helper in training mode.
Creating a new model.


/home/mbugueno/graph-based_TR/TextLevelGCN/wandb_loader_TLGCN.py:290: RuntimeWarning: invalid value encountered in double_scalars
  pmi_matrix[i, j] = np.log(pair_count_matrix[i, j] / (word_count[i] * word_count[j]))
/home/mbugueno/graph-based_TR/TextLevelGCN/wandb_loader_TLGCN.py:290: RuntimeWarning: divide by zero encountered in log
  pmi_matrix[i, j] = np.log(pair_count_matrix[i, j] / (word_count[i] * word_count[j]))


Loading word2vec-google-news-300
Model(
  (seq_edge_w): Embedding(206452, 1)
  (node_hidden): Embedding(1177, 300)
  (dropout): Dropout(p=0.2, inplace=False)
  (activation): ReLU()
  (Linear): Linear(in_features=300, out_features=5, bias=True)
)


LOS PARAMS
RESULTA 561057

Cuda available


...

Epoch:  40 -- Iter: 13800, Train Loss:  0.34, Train Acc:88.97% Val Acc:62.14%, Val_F1ma:34.07%, Time:0:10:39


/home/mbugueno/anaconda3/envs/py38_torch111/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


Epoch:  40 -- Iter: 14000, Train Loss:  0.37, Train Acc:88.34% Val Acc:61.05%, Val_F1ma:32.64%, Time:0:10:48


/home/mbugueno/anaconda3/envs/py38_torch111/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'
/home/mbugueno/anaconda3/envs/py38_torch111/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


Test acc: 0.611
Test F1: [0.492 0.098 0.169 0.141 0.786]
Test F1-ma: 0.337
Graph creation time 2: 53.28979420661926
Running time 2: 663.0957996845245

************************************************
RESULTS FOR N_GRAM: 3 EMB DIM_FEATURES: word2vec-google-news-300
Test Acc: 0.633 -- std: 0.016
Test F1-macro: 0.349 -- std: 0.009
Test F1 per class: [0.513 0.142 0.132 0.159 0.799]
Mean Creation Time: 62.169301668802895
Mean Running Time: 445.4976195494334
Mean Total Execution Time: 507.6669212182363
************************************************




In [9]:
embedding_path="word2vec-google-news-300"
dataset="app_reviews"
project_name="ESC_TLGCN_APP"
file_to_save= "WAND_TextLevelGCN_"+dataset
bs= 16
ng= 4

acc_tests, f1ma_tests, f1_tests = run_loop(project_name, file_to_save, dataset, bs, ng, 3, 100, embedding_path, 300, bert=False)


Running execution 0


wandb: Currently logged in as: mbuguenop (gnn_tc). Use `wandb login --relogin` to force relogin


Loading data helper in training mode.
Creating a new model.


/home/mbugueno/graph-based_TR/TextLevelGCN/wandb_loader_TLGCN.py:290: RuntimeWarning: invalid value encountered in double_scalars
  pmi_matrix[i, j] = np.log(pair_count_matrix[i, j] / (word_count[i] * word_count[j]))
/home/mbugueno/graph-based_TR/TextLevelGCN/wandb_loader_TLGCN.py:290: RuntimeWarning: divide by zero encountered in log
  pmi_matrix[i, j] = np.log(pair_count_matrix[i, j] / (word_count[i] * word_count[j]))


Loading word2vec-google-news-300
Model(
  (seq_edge_w): Embedding(206452, 1)
  (node_hidden): Embedding(1177, 300)
  (dropout): Dropout(p=0.2, inplace=False)
  (activation): ReLU()
  (Linear): Linear(in_features=300, out_features=5, bias=True)
)


LOS PARAMS
RESULTA 561057

Cuda available


...

Epoch:  37 -- Iter: 13000, Train Loss:  0.37, Train Acc:88.03% Val Acc:58.65%, Val_F1ma:36.25%, Time:0:08:31


/home/mbugueno/anaconda3/envs/py38_torch111/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


Epoch:  38 -- Iter: 13200, Train Loss:  0.36, Train Acc:88.53% Val Acc:59.67%, Val_F1ma:35.97%, Time:0:08:39


/home/mbugueno/anaconda3/envs/py38_torch111/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'
/home/mbugueno/anaconda3/envs/py38_torch111/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


Test acc: 0.623
Test F1: [0.488 0.142 0.125 0.165 0.793]
Test F1-ma: 0.343
Graph creation time 2: 57.5808641910553
Running time 2: 532.0581758022308

************************************************
RESULTS FOR N_GRAM: 4 EMB DIM_FEATURES: word2vec-google-news-300
Test Acc: 0.640 -- std: 0.012
Test F1-macro: 0.359 -- std: 0.012
Test F1 per class: [0.503 0.192 0.142 0.152 0.805]
Mean Creation Time: 58.59310587247213
Mean Running Time: 383.59474658966064
Mean Total Execution Time: 442.18785246213275
************************************************


